# Seventh Project: LLVM-IR Code Generation

In this seventh project, you are going to translate the optimized SSA intermediate
representation uCIR into LLVM IR, the intermediate representation of LLVM that is
partially specified in [LLVM Primer](./doc/llvm_primer.ipynb). LLVM is a set of
production-quality reusable libraries for building compilers. LLVM separates computer
architectures from language issues and simplifies the design and portability of new
compilers. Before you start, carefully study this simplified specification of the
LLVM-IR, tailored to our needs, to familiarize yourself with data structures,
addressing modes, and the instructions. The full LLVM IR specifications are available
[here](https://llvm.org/docs/LangRef.html).

We will try to give you a hand by giving you some of the pieces prewritten and
littered your code with helpful statements. By the time you are done, you will have a
pretty thorough understanding of the execution code for uC programs and will even gain
a little reading familiarity with LLVM-IR.

## LLVM python binding

To carry out this step, we will make use of the *llvmlite* library: a lightweight LLVM python 
binding for writing JIT compilers. In fact, we do not need the full LLVM API. Only the IR
builder, optimizer, and JIT compiler APIs are necessary. So, *llvmlite* is a project tailored
for our usage that uses the following approach: a small C wrapper around the parts of the LLVM C++
API we need that are not already exposed by the LLVM C API; a ctypes Python wrapper around
the C API; and a pure Python implementation of the subset of the LLVM IR builder. You can
find the documentation and how to install *llvmlite* at http://llvmlite.pydata.org. An
example, extracted from the *llvmlite* documentation, is presented [here](./doc/llvm_example.ipynb).
Study it carefully.


## Generating Code

The basic idea in this step is similar to the one used by the interpreter. You will
make a class that walks through the CFG, get the sequence of instructions inside the basic block 
and triggers a method for each kind of instruction. However, you will be generating
LLVM instructions instead of running the instructions. The code below show an
example on how to interface with LLVM. Further instructions are contained in the comments.

In [ ]:
from llvmlite import ir, binding
from ctypes import CFUNCTYPE, c_int

class CodeGen():
    def __init__(self):
        self.binding = binding
        self.binding.initialize()
        self.binding.initialize_native_target()
        self.binding.initialize_native_asmprinter()
        
        self.module = ir.Module(name=__file__)
        self.module.triple = self.binding.get_default_triple()
        
        self._create_execution_engine()
        
        # declare external functions
        self._declare_printf_function()
        self._declare_scanf_function()

    def _create_execution_engine(self):
        """
        Create an ExecutionEngine suitable for JIT code generation on
        the host CPU.  The engine is reusable for an arbitrary number of
        modules.
        """
        target = self.binding.Target.from_default_triple()
        target_machine = target.create_target_machine()
        # And an execution engine with an empty backing module
        backing_mod = binding.parse_assembly("")
        engine = binding.create_mcjit_compiler(backing_mod, target_machine)
        self.engine = engine

    def _declare_printf_function(self):
        voidptr_ty = ir.IntType(8).as_pointer()
        printf_ty = ir.FunctionType(ir.IntType(32), [voidptr_ty], var_arg=True)
        printf = ir.Function(self.module, printf_ty, name="printf")
        self.printf = printf

    def _declare_scanf_function(self):
        voidptr_ty = ir.IntType(8).as_pointer()
        scanf_ty = ir.FunctionType(ir.IntType(32), [voidptr_ty], var_arg=True)
        scanf = ir.Function(self.module, scanf_ty, name="scanf")
        self.scanf = scanf

    def _compile_ir(self):
        """
        Compile the LLVM IR string with the given engine.
        The compiled module object is returned.
        """
        # Create a LLVM module object from the IR
        llvm_ir = str(self.module)
        mod = self.binding.parse_assembly(llvm_ir)
        mod.verify()
        # Now add the module and make sure it is ready for execution
        self.engine.add_module(mod)
        self.engine.finalize_object()
        self.engine.run_static_constructors()
        return mod

    def save_ir(self, outputfile):
        output_file.write(str(self.module))
            
    def execute_ir(self, opt, opt_file):
        mod = self._compile_ir()
        
        if opt:
            # apply some optimization passes on module
            pmb = self.binding.create_pass_manager_builder()
            pm = self.binding.create_module_pass_manager()
            
            pmb.opt_level = 0;
            if opt == 'ctm' or opt == 'all':
                # Sparse conditional constant propagation and merging
                pm.add_sccp_pass()
                # Merges duplicate global constants together
                pm.add_constant_merge_pass()
                # Combine inst to form fewer, simple inst
                # This pass also does algebraic simplification
                pm.add_instruction_combining_pass()
            if opt == 'dce' or opt == 'all':
                pm.add_dead_code_elimination_pass()
            if opt == 'cfg' or opt  == 'all':
                # Performs dead code elimination and basic block merging
                pm.add_cfg_simplification_pass()
            
            pmb.populate(pm)
            pm.run(mod)
            opt_file.write(str(mod))
            
        # Obtain a pointer to the compiled 'main' - it's the address of its JITed code in memory.
        main_ptr = self.engine.get_function_address('main')
        # To convert an address to an actual callable thing we have to use
        # CFUNCTYPE, and specify the arguments & return type.
        main_function = CFUNCTYPE(c_int)(main_ptr)
        # Now 'main_function' is an actual callable we can invoke
        res = main_function()
        print(res)

## printf Sample Code 

The code snippet below shows how you can map the print instruction from uCIR to LLVM. Part
of the class structure that generates LLVM instructions for a function is shown. Use it as
a guide.

In [ ]:
def make_bytearray(buf):
    # Make a byte array constant from *buf*.
    b = bytearray(buf)
    n = len(b)
    return ir.Constant(ir.ArrayType(ir.IntType(8), n), b)


class LLVMFunctionVisitor(BlockVisitor):

    def __init__(self, module):
        self.module = module
        self.func = None
        self.builder = None
        self.loc = {}

    def _global_constant(self, builder_or_module, name, value, linkage='internal'):
        # Get or create a (LLVM module-)global constant with *name* or *value*.
        if isinstance(builder_or_module, ir.Module):
            mod = builder_or_module
        else:
            mod = builder_or_module.module
        data = ir.GlobalVariable(mod, value.type, name=name)
        data.linkage = linkage
        data.global_constant = True
        data.initializer = value
        data.align = 1
        return data

    def _cio(self, fname, format, *target):
        # Make global constant for string format
        mod = self.builder.module
        fmt_bytes = make_bytearray((format + '\00').encode('ascii'))
        global_fmt = self._global_constant(mod, mod.get_unique_name('.fmt'), fmt_bytes)
        fn = mod.get_global(fname)
        ptr_fmt = self.builder.bitcast(global_fmt, ir.IntType(8).as_pointer())
        return self.builder.call(fn, [ptr_fmt] + list(target))

    def _build_print(self, val_type, target):
        if target:
            # get the object assigned to target
            _value = self._get_loc(target)
            if val_type == 'int':
                self._cio('printf', '%d', _value)
            elif val_type == 'float':
                self._cio('printf', '%.2f', _value)
            elif val_type == 'char':
                self._cio('printf', '%c', _value)
            elif val_type == 'string':
                self._cio('printf', '%s', _value)
        else:
            self._cio('printf', '\n')

## Differences between uCIR and llvm IR

### Accessing an array element

Some uCIR and llvm IR code generation strategies are different. Be the example below where
access to 1 element of matrix ```v``` is made. While uCIR visualizes the linearized matrix,
computing the index of the element ```v[i][j]``` (```index = i * ncol + j```), llvm IR
first accesses the row vector ```i``` and then the column element ```j``` in this vector,
generating with this two ```getelementptr``` instructions. One strategy that can be used
to translate from uCIR to llvm IR is to recalculate indices ```i``` and ```j``` from the
calculated ```index```, based on the number of columns (```ncol```) that is known at
compile time. The redundant code can then be eliminated by the llvm DCE.

```
int main() {
    int v[][] = { {1,2}, {3,4}, {5,6} };
    int x, i, j;
    x = v[i][j];
    return 0;
}
```
**uCIR** without opt (note that the code does not run. variables i & j was not initialized):
```
@.const_v.0 = global int[3][2] [[1, 2], [3, 4], [5, 6]]

define int @main ()
entry:
  %1 = alloc int 
  %v = alloc int[3][2] 
  %x = alloc int 
  %i = alloc int 
  %j = alloc int 
  store int[3][2] @.const_v.0 %v 
  %2 = literal int 2 
  %3 = load int %i 
  %4 = mul int %2 %3 
  %5 = load int %j 
  %6 = add int %4 %5 
  %7 = elem int %v %6 
  %8 = load int* %7 
  store int %8 %x 
  %9 = literal int 0 
  store int %9 %1 
  jump label %exit
exit:
  %10 = load int %1 
  return int %10
```
**llvm IR**:
```
; ModuleID = "/Users/marcio/PycharmProjects/mc921/uc/uc_llvm.py"
target triple = "x86_64-apple-darwin19.6.0"
target datalayout = ""

declare i32 @"printf"(i8* %".1", ...) 

declare i32 @"scanf"(i8* %".1", ...) 

@".const_v.0" = constant [3 x [2 x i32]] [[2 x i32] [i32 1, i32 2], [2 x i32] [i32 3, i32 4], [2 x i32] [i32 5, i32 6]], align 16
define i32 @"main"() 
{
entry:
  %"1" = alloca i32, align 4
  %"v" = alloca [3 x [2 x i32]], align 16
  %"x" = alloca i32, align 4
  %"i" = alloca i32, align 4
  %"j" = alloca i32, align 4
  %".2" = bitcast [3 x [2 x i32]]* @".const_v.0" to i8*
  %".3" = bitcast [3 x [2 x i32]]* %"v" to i8*
  call void @"llvm.memcpy.p0i8.p0i8.i64"(i8* %".3", i8* %".2", i64 24, i1 false)
  %".5" = load i32, i32* %"i", align 4
  %".6" = mul i32 2, %".5"
  %".7" = load i32, i32* %"j", align 4
  %".8" = add i32 %".6", %".7"
  %".9" = sdiv i32 %".8", 2
  %".10" = srem i32 %".8", 2
  %".11" = getelementptr [3 x [2 x i32]], [3 x [2 x i32]]* %"v", i32 0, i32 %".9"
  %".12" = getelementptr [2 x i32], [2 x i32]* %".11", i32 0, i32 %".10"
  %".13" = load i32, i32* %".12", align 4
  store i32 %".13", i32* %"x", align 4
  store i32 0, i32* %"1", align 4
  br label %"exit"
exit:
  %".17" = load i32, i32* %"1", align 4
  ret i32 %".17"
}

declare void @"llvm.memcpy.p0i8.p0i8.i64"(i8* %".1", i8* %".2", i64 %".3", i1 %".4") 
```

### Pointer arithmetic

Accessing the content of pointer variables performed by the ```load_type_*``` instruction
in uCIR requires two ```load``` instructions in llvm IR to obtain dereferencing. See the
example below:

```
int main() {
    int i, *r;
    i = *r;
    return 0;
}
```
**uCIR** with opt on:
```
define int @main ()
entry:
  %1 = alloc int 
  %i = alloc int 
  %r = alloc int* 
  %2 = load int* %r 
  store int %2 %i 
  %3 = literal int 0 
  store int %3 %1 
  jump label %exit
exit:
  %4 = load int %1 
  return int %4
```
**llvm IR**:
```
; ModuleID = "/Users/marcio/PycharmProjects/mc921/uc/uc_llvm.py"
target triple = "x86_64-apple-darwin19.6.0"
target datalayout = ""

declare i32 @"printf"(i8* %".1", ...) 

declare i32 @"scanf"(i8* %".1", ...) 

define i32 @"main"() 
{
entry:
  %"1" = alloca i32, align 4
  %"i" = alloca i32, align 4
  %"r" = alloca i32*, align 8
  %".2" = load i32*, i32** %"r", align 8
  %".3" = load i32, i32* %".2", align 4
  store i32 %".3", i32* %"i", align 4
  store i32 0, i32* %"1", align 4
  br label %"exit"
exit:
  %".7" = load i32, i32* %"1", align 4
  ret i32 %".7"
}
```

Likewise, the ```store_type_*``` statement may require a ```load``` followed by a ```store``` on llvm:

```
int main() {
    int i, *r;
    *r = i;
    return 0;
}
```
**uCIR** (without opt):
```
define int @main ()
entry:
  %1 = alloc int 
  %i = alloc int 
  %r = alloc int* 
  %2 = load int %i 
  store int* %2 %r 
  %3 = literal int 0 
  store int %3 %1 
  jump label %exit
exit:
  %4 = load int %1 
  return int %4
```
**llvm IR**:
```
; ModuleID = "/Users/marcio/PycharmProjects/mc921/uc/uc_llvm.py"
target triple = "x86_64-apple-darwin19.6.0"
target datalayout = ""

declare i32 @"printf"(i8* %".1", ...) 

declare i32 @"scanf"(i8* %".1", ...) 

define i32 @"main"() 
{
entry:
  %"1" = alloca i32, align 4
  %"i" = alloca i32, align 4
  %"r" = alloca i32*, align 8
  %".2" = load i32, i32* %"i", align 4
  %".3" = load i32*, i32** %"r"
  store i32 %".2", i32* %".3"
  store i32 0, i32* %"1", align 4
  br label %"exit"
exit:
  %".7" = load i32, i32* %"1", align 4
  ret i32 %".7"
}
```